<a href="https://colab.research.google.com/github/KwonDoRyoung/AdvancedBasicEducationProgram/blob/main/abep08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.activation function: sigmoid, relu, softmax

In [ ]:
import torch

torch.cuda.is_available()

In [ ]:
import torch.nn as nn # neural network
import torch.nn.functional as F 
# neural network 기능 중에 함수형태로 프로그래밍된 친구들

class SingleLayerPerceptron(nn.Module): 
  # torch에서는 역전파를 제공합니다.
  # 선언한 class 에 nn.Module 이라는 class 상속 시키면.
  # 상속: 미리 누군가 정의해둔 청사진 > 청사진을 토대로 제가 활용하는 것
  # 알아서 backpropagation 정의됩니다.
  def __init__(self,):
    super(SingleLayerPerceptron, self).__init__() # <- 상속 하겠다.
    # super.__init__() # or super(SingleLayerPerceptron, self).__init__()
    # 모델에 구조를 선언하는 부분
    self.fc = nn.Linear(784, 10, bias=False) # mnist 1,28,28 -> 784 펼쳤고, 맞춰야될 정답이 0~9 -> 10개
    # 10개를 하나의 출력으로 맞추는 것은 힘들기 때문에
    # 10개의 출력에 대한 확률을 구하고 거기서 제일 높은 확률을 가지는 인덱스를 정답으로 예측한다.
    # 제일 높은 확률을 만들기 위해서는 softmax를 활용하고
    # 비용함수는 cross entropy 활용한다.
  
  def forward(self, inputs):
    # 입력받는 부분을 정의하는 곳
    # 출력 내는 곳
    outputs = self.fc(inputs)
    return outputs


In [ ]:

import torchvision
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt

import torch.utils
t = torchvision.transforms.Compose([torchvision.transforms.PILToTensor()])
dataset = MNIST(root="/content/mnist", download=True, transform=t)
# data ->index 0: image, 1: target 
# [[<pil.image class, 상수_target>] -> tensor
# [<pil.image class, 상수_target>]
# [<pil.image class, 상수_target>]
# [<pil.image class, 상수_target>]]
batch_size = 32
data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

criterion = nn.CrossEntropyLoss()
# 비용함수를 만드는 이유는 최적화.
# 최소화 

model = SingleLayerPerceptron().to("cuda") # gpu에서 할거면 model, data 모든게 Gpu에 할당되야됨

optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

end_epoch =100
for epoch in range(end_epoch):
  mean_cost = 0
  count = 0
  for step, (batch_image, batch_target) in enumerate(data_loader):
    batch_image = torch.flatten(batch_image, start_dim=1).float().to("cuda")
    batch_target = batch_target.to("cuda")
    predicts = model(batch_image)

    cost = criterion(predicts, batch_target)

    optimizer.zero_grad() # 이전에 계산한 미분값 지우는 것
    cost.backward() # 현재 데이터에 대한 미분값 계산
    optimizer.step() # 현재 데이터로 만든 미분값으로 최적화를 해라.
    
    mean_cost += cost.item()
    count += 1
  if epoch % 5 == 0:
    print(f"[Epoch]: {epoch}, Loss: {mean_cost/count}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn

In [ ]:
sigmoid = nn.Sigmoid()
relu = nn.ReLU()
softmax = nn.Softmax()

In [ ]:
''' 1. sigmoid, rleu 그래프 그리기'''
x = np.linspace(-7, 7, 100)
y = sigmoid(torch.Tensor([x]))[0,:]
z = relu(torch.Tensor([x]))[0,:]
plt.figure(figsize=(20,10))
plt.subplot(2, 2, 1)
plt.plot(x, y, color='green')
plt.title('sigmoid')
plt.subplot(2, 2, 2)
plt.plot(x, z, color='steelblue')
plt.title('relu')
plt.show()

In [ ]:
''' 2. softmax 특징'''
print(softmax(torch.Tensor([2,3,4])))
print(softmax(torch.Tensor([2,3,4])).sum())

2.perceptron torch구현

In [ ]:
import torch
import torch.nn as nn

In [ ]:
EPOCHS = 300

In [ ]:
''' AND_gate 문제'''
input = torch.FloatTensor([[0,0],[0,1],
                           [1,0],[1,1]])
target = torch.LongTensor([0,0,0,1])

In [ ]:
''' 1. 딥러닝 모델을 설계할 때 활용하는 장비 확인 '''
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

In [ ]:
''' 2. Perceptron 모델 설계하기 '''
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(2,2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc(x)
        return x

In [ ]:
''' 3. Optimizer, Objective Function 설정하기 '''
model = Perceptron().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [ ]:
''' 4. 모델 학습을 진행하며 학습 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def train(model, input, target, optimizer, log_interval):
    model.train()
    input = input.to(DEVICE)
    target = target.to(DEVICE)
    optimizer.zero_grad()
    output = model(input)
    loss = criterion(output,target)
    loss.backward()
    optimizer.step()

In [ ]:
''' 5. 학습되는 과정 속에서 검증 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def evaluate(model, input, target):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        input = input.to(DEVICE)
        target = target.to(DEVICE)
        output = model(input)
        test_loss += criterion(output, target).item()
        prediction = output.max(1, keepdim = True)[1]
        correct += prediction.eq(target.view_as(prediction)).sum().item()
    test_accuracy = 100. * correct / 4
    return test_loss, prediction, test_accuracy

In [ ]:
''' 6. MLP 학습 실행하며 Train, Test set의 Loss 및 Test set Accuracy 확인하기 '''
for epoch in range(1, EPOCHS + 1):
    train(model, input, target, optimizer, log_interval = 200)
    test_loss, prediction, test_accuracy = evaluate(model, input, target)
    if (epoch)%10 == 0:
        print("\n[EPOCH: {}], \toutput: {}, \ttarget: {}, \tLoss: {}, \tTest Accuracy: {:.2f} %".format(
            epoch, prediction.view(4).tolist(), target.tolist(), test_loss, test_accuracy))

3.multi-layer perceptron torch 구현
MNIST

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
BATCH_SIZE = 32
EPOCHS = 5

In [ ]:
''' 1. 딥러닝 모델을 설계할 때 활용하는 장비 확인 '''
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

In [ ]:
train_path = '/content/drive/MyDrive/Colab Notebooks/MNIST/mnist_train.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/MNIST/mnist_test.csv'

In [ ]:
class CustomDataset1(Dataset):
    def __init__(self,path):
        data = pd.read_csv(path)
        X = data.drop(['label'],axis=1)
        X = X.values.reshape(-1,1,28,28)
        X = torch.FloatTensor(X)
        y = data['label']
        y = y.values.reshape(-1)
        y = torch.LongTensor(y)
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self,idx):
        a = self.X[idx]
        b = self.y[idx]
        return a,b

In [ ]:
''' 2. Dataset & Dataloader 사용하여 Custom Dataset 만들기 '''
train_dataset = CustomDataset1(train_path)
test_dataset = CustomDataset1(test_path)
train_loader = DataLoader(dataset = train_dataset,
                          batch_size = BATCH_SIZE,
                          shuffle = True)
test_loader = DataLoader(dataset = test_dataset,
                         batch_size = BATCH_SIZE,
                         shuffle = False)

In [ ]:
''' 3. 데이터 확인하기(1) '''
for (image, label) in train_loader:
    print('image:', image.size(), 'type:', image.type())
    print('label:', label.size(), 'type:', label.type())
    break
    ''' 4. 데이터 확인하기 (2) '''
pltsize = 1
plt.figure(figsize=(10 * pltsize, pltsize))
for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.axis('off')
    plt.imshow(image[i, :, :, :].numpy().reshape(28, 28), cmap = "gray_r")
    plt.title('Class: ' + str(label[i].item()))

In [ ]:
''' 5. Multi Layer Perceptron (MLP) 모델 설계하기 '''
class MLP1(nn.Module):
    def __init__(self):
        super(MLP1, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [ ]:
''' 6. Optimizer, Objective Function 설정하기 '''
model = MLP1().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
''' 7. MLP 모델 학습을 진행하며 학습 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [ ]:
''' 8. 학습되는 과정 속에서 검증 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= (len(test_loader.dataset) / BATCH_SIZE)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
''' 9. MLP 학습 실행하며 Train, Test set의 Loss 및 Test set Accuracy 확인하기 '''
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

4.multi-layer perceptron torch 구현
Fashion-MNIST

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
BATCH_SIZE = 32
EPOCHS = 5

In [ ]:
''' 1. 딥러닝 모델을 설계할 때 활용하는 장비 확인 '''
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

In [ ]:
train_path = '/content/drive/MyDrive/Colab Notebooks/Fashion mnist/fashion-mnist_train.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/Fashion mnist/fashion-mnist_test.csv'

In [ ]:
class CustomDataset2(Dataset):
    def __init__(self,path):
        data = pd.read_csv(path)
        X = data.drop(['label'],axis=1)
        X = X.values.reshape(-1,1,28,28)
        X = torch.FloatTensor(X)
        y = data['label']
        y = y.values.reshape(-1)
        y = torch.LongTensor(y)
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self,idx):
        a = self.X[idx]
        b = self.y[idx]
        return a,b

In [ ]:
''' 2. Dataset & Dataloader 사용하여 Custom Dataset 만들기 '''
train_dataset = CustomDataset2(train_path)
test_dataset = CustomDataset2(test_path)
train_loader = DataLoader(dataset = train_dataset,
                          batch_size = BATCH_SIZE,
                          shuffle = True)
test_loader = DataLoader(dataset = test_dataset,
                         batch_size = BATCH_SIZE,
                         shuffle = False)

In [ ]:
''' 3. 데이터 확인하기(1) '''
for (image, label) in train_loader:
    print('image:', image.size(), 'type:', image.type())
    print('label:', label.size(), 'type:', label.type())
    break

In [ ]:
''' 4. 데이터 확인하기 (2) '''
pltsize = 1
plt.figure(figsize=(10 * pltsize, pltsize))
for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.axis('off')
    plt.imshow(image[i, :, :, :].numpy().reshape(28, 28), cmap = "gray_r")
    plt.title('Class: ' + str(label[i].item()))

In [ ]:
''' 5. Multi Layer Perceptron (MLP) 모델 설계하기 '''
class MLP2(nn.Module):
    def __init__(self):
        super(MLP2, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [ ]:
''' 6. Optimizer, Objective Function 설정하기 '''
model = MLP2().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
''' 7. MLP 모델 학습을 진행하며 학습 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [ ]:
''' 8. 학습되는 과정 속에서 검증 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= (len(test_loader.dataset) / BATCH_SIZE)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
''' 9. MLP 학습 실행하며 Train, Test set의 Loss 및 Test set Accuracy 확인하기 '''
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

5.multi-layer perceptron torch 구현
CIFAR_10

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
BATCH_SIZE = 32
EPOCHS = 5

In [ ]:
''' 1. 딥러닝 모델을 설계할 때 활용하는 장비 확인 '''
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

In [ ]:
train_path = "/content/drive/MyDrive/Colab Notebooks/cifar-10/cifar-10_train.csv"
test_path = '/content/drive/MyDrive/Colab Notebooks/cifar-10/cifar-10_test.csv'

In [ ]:
class CustomDataset3(Dataset):
    def __init__(self,path):
        data = pd.read_csv(path)
        X = data.drop(['label'],axis=1)/255
        X = X.values.reshape(-1,3,32,32)
        X = torch.FloatTensor(X)
        y = data['label']
        y = y.values.reshape(-1)
        y = torch.LongTensor(y)
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self,idx):
        a = self.X[idx]
        b = self.y[idx]
        return a,b

In [ ]:
''' 2. Dataset & Dataloader 사용하여 Custom Dataset 만들기 '''
train_dataset = CustomDataset3(train_path)
test_dataset = CustomDataset3(test_path)
train_loader = DataLoader(dataset = train_dataset,
                          batch_size = BATCH_SIZE,
                          shuffle = True)
test_loader = DataLoader(dataset = test_dataset,
                         batch_size = BATCH_SIZE,
                         shuffle = False)

In [ ]:
''' 3. 데이터 확인하기(1) '''
for (image, label) in train_loader:
    print('image:', image.size(), 'type:', image.type())
    print('label:', label.size(), 'type:', label.type())
    break

In [ ]:
''' 4. 데이터 확인하기 (2) '''
pltsize = 1
plt.figure(figsize=(10 * pltsize, pltsize))
for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.axis('off')
    plt.imshow(np.transpose(image[i], (1, 2, 0)))
    plt.title('Class: ' + str(label[i].item()))

In [ ]:
''' 5. Multi Layer Perceptron (MLP) 모델 설계하기 '''
class MLP3(nn.Module):
    def __init__(self):
        super(MLP3, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 2048)
        self.fc2 = nn.Linear(2048,1024)
        self.fc3 = nn.Linear(1024,512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 32 * 32 * 3)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.relu(x)
        x = self.fc5(x)
        return x

In [ ]:
''' 6. Optimizer, Objective Function 설정하기 '''
model = MLP3().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
criterion = nn.CrossEntropyLoss()

In [ ]:
''' 7. MLP 모델 학습을 진행하며 학습 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [ ]:
''' 8. 학습되는 과정 속에서 검증 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= (len(test_loader.dataset) / BATCH_SIZE)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
''' 9. MLP 학습 실행하며 Train, Test set의 Loss 및 Test set Accuracy 확인하기 '''
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))